In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import LSTM, Dense, Dropout
from sklearn.preprocessing import MinMaxScaler
from sqlalchemy import create_engine, inspect

In [ ]:
# Database connection
DATABASE_URL = "your_database_url_here"
engine = create_engine(DATABASE_URL)

In [ ]:
# Get all table names (assuming each table is a stock ticker)
def get_stock_tables():
    inspector = inspect(engine)
    return inspector.get_table_names()

In [ ]:
# Load stock data from table
def load_stock_data(ticker):
    query = f"SELECT * FROM {ticker} ORDER BY date"
    df = pd.read_sql(query, con=engine)
    df['date'] = pd.to_datetime(df['date'])
    df.set_index('date', inplace=True)
    
    # Ensure necessary columns exist
    if 'close' not in df.columns:
        raise ValueError(f"Table {ticker} does not contain a 'close' column.")

    # Select features (close + any technical indicators if available)
    features = [col for col in df.columns if col != 'date']
    return df[features]

In [ ]:
# Prepare dataset for LSTM
def prepare_data(df, lookback=30):
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_data = scaler.fit_transform(df)

    X, y = [], []
    for i in range(len(scaled_data) - lookback - 30):
        X.append(scaled_data[i:i+lookback])   # Last 30 days of data
        y.append(scaled_data[i+lookback:i+lookback+30, 0])  # Predict next 30 days of close price

    return np.array(X), np.array(y), scaler

In [ ]:
# Define or load LSTM model
def build_or_load_model(input_shape, model_path="lstm_model.h5"):
    try:
        model = load_model(model_path)
        print("Loaded existing model.")
    except:
        print("Creating new model...")
        model = Sequential([
            LSTM(100, return_sequences=True, input_shape=input_shape),
            Dropout(0.2),
            LSTM(50, return_sequences=False),
            Dropout(0.2),
            Dense(30)  # Predict next 30 days of prices
        ])
        model.compile(optimizer='adam', loss='mse')

    return model

In [ ]:
# Train model on multiple tickers
def train_model():
    tickers = get_stock_tables()
    model = None

    for ticker in tickers:
        print(f"Training on {ticker}...")
        df = load_stock_data(ticker)
        X, y, scaler = prepare_data(df)

        train_size = int(len(X) * 0.8)
        X_train, X_test = X[:train_size], X[train_size:]
        y_train, y_test = y[:train_size], y[train_size:]

        if model is None:
            model = build_or_load_model((X.shape[1], X.shape[2]))

        model.fit(X_train, y_train, epochs=30, batch_size=32, validation_data=(X_test, y_test), verbose=1)

    # Save model after training on all tickers
    model.save("lstm_model.h5")
    print("Model saved.")

In [ ]:
# Predict next 30 days for each stock
def predict_next_30_days():
    tickers = get_stock_tables()
    model = load_model("lstm_model.h5")

    predictions = {}
    for ticker in tickers:
        print(f"Predicting for {ticker}...")
        df = load_stock_data(ticker)
        X, _, scaler = prepare_data(df)

        last_sequence = X[-1].reshape(1, X.shape[1], X.shape[2])
        predicted_scaled = model.predict(last_sequence)[0]
        predicted_prices = scaler.inverse_transform(predicted_scaled.reshape(-1, 1))

        predictions[ticker] = predicted_prices.flatten()

    return predictions

In [ ]:
# Run training & prediction
train_model()
predictions = predict_next_30_days()

In [ ]:
# Print predictions
for ticker, prices in predictions.items():
    print(f"{ticker} Next 30 Days Predictions:", prices)